In [1]:
%load_ext autoreload
%autoreload 2

# IEX Stock Fetcher

The IEX free message quota only gets one year of data for 200 stocks, so run this, fetcher stock data and save to disk every month.


## Test api

In [49]:
import iex
iex.init('Tpk_8dad7c6b1806466dbb3c1b3c07fa5be1', api='sandbox')
aapl = iex.Stock('AAPL')
aapl.chart('5d')

,change,changeOverTime,changePercent,close,high,label,low,open,uClose,uHigh,uLow,uOpen,uVolume,volume
date,,,,,,,,,,,,,,
2019-09-30,0.00,0.000000,0.0000,233.55,232.94,Sep 30,221.48,224.00,228.49,226.79,229.87,221.00,26324314,26990686
2019-10-01,0.64,0.002867,0.2789,231.94,236.72,Oct 1,231.30,226.46,226.77,237.37,232.20,228.45,37300050,37740641
2019-10-02,-5.70,-0.022807,-2.6285,223.67,231.00,Oct 2,226.87,223.61,219.79,233.90,222.10,233.27,37190954,36897959
2019-10-03,1.95,-0.014448,0.8510,229.09,224.64,Oct 3,223.64,222.83,228.46,224.76,222.09,220.24,31829034,31639591
2019-10-04,6.22,0.014130,2.8757,230.11,232.69,Oct 4,229.78,235.12,236.55,227.84,228.39,231.80,36212147,35343376


## Fetche SPY stocks

In [51]:
import pandas as pd
spy = [x for x in pd.read_html('https://etfdailynews.com/etf/spy/', attrs={'id': 'etfs-that-own'})[0].Symbol.values.tolist() if isinstance(x, str)]

In [ ]:
# iex.init('pk_xxx', api='cloud')
iex.init('Tpk_8dad7c6b1806466dbb3c1b3c07fa5be1', api='sandbox')
for k in spy:
    # todo read csv and add needed range
    iex.Stock(k).chart('1y').to_csv("./daily/{}.csv".format(k))